In [31]:
import pandas as pd
import numpy as np

drivers = pd.read_csv("drivers.csv")

redbull_drivers = (
    drivers[drivers["team_name"].str.contains("Red Bull", case=False, na=False)]
    .drop(columns=["headshot_url"])
    .reset_index(drop=True)
)

redbull_drivers.to_csv("redbull_drivers.csv", index=False)
print("done")

done


In [32]:
redbull_drivers = pd.read_csv("redbull_drivers.csv")
stints = pd.read_csv("stints.csv")

rb_numbers = redbull_drivers["driver_number"].unique()
rb_stints = stints[stints["driver_number"].isin(rb_numbers)].reset_index(drop=True)

rb_stints.to_csv("rb_stints.csv", index=False)
print("done")

done


In [33]:
sessions = pd.read_csv("sessions.csv")
rb_stints = pd.read_csv("rb_stints.csv")

rb_stints = rb_stints.merge(
    sessions[["session_key", "year", "session_type"]],
    on="session_key",
    how="left"
)

rb_stints = rb_stints[
    (rb_stints["year"].notna()) &
    (rb_stints["year"] != 2026) &
    (rb_stints["session_type"] == "Race")
]

rb_stints = rb_stints.drop(columns=["year", "session_type"]).reset_index(drop=True)

rb_stints.to_csv("rb_stints.csv", index=False)
print("done")

done


In [72]:
bad_dates = pd.to_datetime([
    "2023-04-29",
    "2023-07-29",
    "2023-10-07",
    "2023-10-21",
    "2023-11-04",
    "2024-04-20",
    "2024-05-04",
    "2024-06-29",
    "2024-10-19",
    "2024-11-02",
    "2024-11-30",
    "2025-03-22",
    "2025-05-03",
    "2025-07-26",
    "2025-10-18",
    "2025-11-08",
    "2025-11-29",
]).date

tmp = rb_stints.merge(
    sessions[["session_key", "date_start"]],
    on="session_key",
    how="left"
)

tmp["date_start"] = pd.to_datetime(tmp["date_start"], errors="coerce").dt.date

filtered = tmp[~tmp["date_start"].isin(bad_dates)] \
            .drop(columns=["date_start"]) \
            .reset_index(drop=True)

filtered.to_csv("rb_stints.csv", index=False)


In [77]:
rb_stints = pd.read_csv("rb_stints.csv")

rb_stints["lap_start"] = pd.to_numeric(rb_stints["lap_start"], errors="coerce")
rb_stints["lap_end"]   = pd.to_numeric(rb_stints["lap_end"], errors="coerce")

rb_stints["lap_end_duration"] = np.nan
rb_stints["fastest_lap_duration"] = np.nan

groups = rb_stints.groupby(["driver_number", "session_key"]).groups

for (driver_number, session_key), idx in groups.items():
    driver_number = int(driver_number)
    session_key = int(session_key)

    url = f"https://api.openf1.org/v1/laps?driver_number={driver_number}&session_key={session_key}"
    laps = pd.read_json(url)

    if laps is None or laps.empty:
        continue

    laps["lap_number"] = pd.to_numeric(laps["lap_number"], errors="coerce")
    laps["lap_duration"] = pd.to_numeric(laps["lap_duration"], errors="coerce")

    for i in idx:
        lap_start = rb_stints.at[i, "lap_start"]
        lap_end = rb_stints.at[i, "lap_end"]

        if pd.isna(lap_start) or pd.isna(lap_end):
            continue

        lap_start = int(lap_start)
        lap_end = int(lap_end)

        stint_laps = laps[(laps["lap_number"] >= lap_start) & (laps["lap_number"] <= lap_end)]
        if stint_laps.empty:
            continue

        rb_stints.at[i, "fastest_lap_duration"] = stint_laps["lap_duration"].min(skipna=True)

        end_row = stint_laps[stint_laps["lap_number"] == lap_end]
        if not end_row.empty:
            rb_stints.at[i, "lap_end_duration"] = end_row["lap_duration"].iloc[0]

rb_stints.to_csv("rb_stints.csv", index=False)

print("Updated rb_stints.csv")
print("Rows with missing lap_start or lap_end:", rb_stints["lap_start"].isna().sum() + rb_stints["lap_end"].isna().sum())


Updated rb_stints.csv
Rows with missing lap_start or lap_end: 18


In [78]:
rb_stints = rb_stints.dropna(subset=["lap_start", "lap_end"]).reset_index(drop=True)

rb_stints.to_csv("rb_stints.csv", index=False)